# Study Local Outlier Factor
Local Outlier Factor

这个算法是基于数据密度进行检测的。首先给定定义

<font color=red>k-distance</font>: 距离数据点p最近的几个点中，第k个最近的点跟点p之间的距离。这个是以给定数量的周边数据点作为边界，算出来的"半径"。  
<font color=red>reachability distance</font>：可达距离。与k-distance相关，给定参数k，p到o的可达距离为o的k-distance与p与o直线距离的最大值，即
$$
reachdist_k(p,o) = \max\{ k-distance(o), d(p,o) \}
$$
我们关注的是点p，它到o的可达距离就是直线距离，但是如果这两个点太近了，那么就以目标点o的"k半径"为准。直观的理解，就是如果点p要到达点o，就是直线距离，但是距离最低是点o的半径。  
<font color=red>local reachability density</font>：局部可达密度，对于与数据点p的距离小于等于k-distance(p)的数据点，就叫做k-nearest-neighbor。记为$N_k(p)$。数据点p的局部可达密度，指的是它与临近的数据点的平均可达距离的倒数，即
$$
lrd_k(p) = \frac{1}{\frac{\sum_{o \in N_k(p) }reachdist_k(p,o)}{|N_k(p)|}}
$$
直观理解就是对于点p而言，其周围的一系列点o，每个点o都是各自的k半径，那么平均可达距离的倒数就是点p的局部可达密度。想象一下，点p如果是比较孤立的，其周围最近的k个点是聚集在一起的，那么reach dist应该就比较大，而$lrd_k(p)$就应该比较小。如果点p是和周围最近的k个点聚集在一起的，那么整体来看reach dist就应该比较小,$lrd_k(p)$就比较大。  
<font color=red>local outlier factor</font>：局部异常因子。上面我们已经能算出来每个点的局部可达密度了，不过我们这里专注的不是绝对密度，而是相对密度。这样做的好处就是可以在数据分布不均匀的情况下，也能正常处理数据，不会把一大片数据都看成outlier。
$$
LOF_k(p) = \frac{\sum_{o \in N_k(p)} \frac{lrd(o)}{lrd(p)}}{|N_k(p)|} = \frac{\sum_{o \in N_k(p)} lrd(o)}{|N_k(p)|} \frac{1}{lrd(p)}
$$
我们可以理解为，数据点p周围的点相对于p的平均密度。这件事的核心在于当前数据点p的"周围"与其他数据点o的"周围"的范围并不一样，o在p的周围，p不一定在o的周围，只是以一个数量k作为标准。所以，这里就体现出了点p周围的点相对于点p的相对密度的概念。相对密度越大，体现出来的是周围的点o的密度相比于点p是不是更加密集，如果相比之下非常稠密，则意味着当前点p是一个孤立点。


![图 1](StudyLocalOutlierFactor/8b58e902b7e799ce28c6da525be8ebfbbc23ef767a1f564df188a4167da32fec.png)  


如图所示，越孤立的点，其LOF越高，我们只要算出来LOF的值，再给个阈值就能把所有的孤立点给挑出来。

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from Common.DataCenter import data_center

现在构造一部分数据点

In [2]:
X = [[-1.1], [0.2], [101.1], [0.3]]
clf = LocalOutlierFactor(n_neighbors=2)
clf.fit_predict(X)

array([ 1,  1, -1,  1])

我们能看得出来，最大的那个101.1被认为是outlier，看起来很不错，现在使用我们的数据

In [12]:
from Common.DataCenter import data_center
from Common.preprocessor import normalize_preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [5]:
train_distribution = None
#Each entry: source type and (size, distribution)
noisy_set_sizes = {
    'mislabeled' : (8000, None),                   # max size: 15000
    'irrelevant' : (2000, [0.25,0.25,0.25,0.25]),  # max size: 34259
    'translated' : (2000, "reserve_labels"),       # max size: 5000
}
dc = data_center("twitter_sentiment_data_clean.csv", train_size = 20000, test_size = 4000, validation_size = 1000,
                 noisy_size = noisy_set_sizes)

trainDF = dc.get_train_with_noisy_df(4000, 1000, train_distribution)
trainDF.reset_index(drop=True, inplace=True)



下面进行去噪

In [7]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train):
    # preprocessing with traditional NLP methodology
    X_train_normalized = normalize_preprocessing(X_train)
    
    # vectorization
    # Convert texts to vectors by TFIDF
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    X_train_vec  = vectorizer.fit_transform(X_train_normalized)
    return X_train_vec
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    return y_train_vec

In [13]:
X_train, y_train = data_center.Xy(trainDF)
X_train_vec = text_preprocessing(X_train)
y_train_vec = one_hot_encoding(y_train)

In [18]:
clf = LocalOutlierFactor(n_neighbors=2)
label_result = clf.fit_predict(X_train_vec)
label_result = np.array(label_result)

In [21]:
label_result[label_result == -1].shape

(34,)

这个方法比EllipticalEnvelope好多了，至少不要求dense matrix